In [71]:
import numpy as np
import pandas as pd
import os.path

root_path = os.path.dirname(os.getcwd())

# Load business license data
licenses = pd.read_csv(os.path.join(root_path, "DATA/business_licenses.csv"))

# Load food inspection data
inspections = pd.read_csv(os.path.join(root_path, "DATA/food_inspections.csv"))

# Load violation data
values = pd.read_csv(os.path.join(root_path, "DATA/violation_values.csv"))
counts = pd.read_csv(os.path.join(root_path, "DATA/violation_counts.csv"))

/home/russell/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,6,23,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### CALCULATE FEATURES BASED ON FOOD INSPECTION DATA

In [72]:
# Create basis for model_data (risk not included in Chicago repository)
data = inspections[["inspection_date", "license", "inspection_id", "facility_type", "results", "risk"]]

In [73]:
# Merge with violation data
data = pd.merge(data, values, on="inspection_id")
data = pd.merge(data, counts, on="inspection_id")

In [74]:
# Create pass / fail flags
pass_flags = data.results.map(lambda x: 1 if x == "Pass" else 0)
fail_flags = data.results.map(lambda x: 1 if x == "Fail" else 0)
data["pass_flags"], data["fail_flags"] = pass_flags, fail_flags

In [75]:
# Sort inspections by date
data.sort_values(by="inspection_date", inplace=True)

# Find previous inspection by shifting columns (grouped by license)
data["past_inspection_id"] = data.groupby("license").inspection_id.shift(1)

In [78]:
# Add previous violation values
data = pd.merge(data, values.add_prefix("past_"), on="past_inspection_id")

In [79]:
# Add previous violation counts
data = pd.merge(data, counts.add_prefix("past_"), on="past_inspection_id")

In [108]:
# Join data with self to find past inspection dates
past_dates = data[["inspection_id", "inspection_date"]].add_prefix("past_")
data = pd.merge(data, past_dates, on="past_inspection_id")

# Calculate time since previous inspection
deltas = pd.to_datetime(temp_data.inspection_date) - pd.to_datetime(temp_data.past_inspection_date)
data["time_since_last"] = deltas.apply(lambda x: x.days / 365.25)

# Drop past inspection date
data.drop("past_inspection_date", inplace=True)

In [109]:
data.time_since_last

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
5       0.000000
6       0.000000
7       0.000000
8       0.000000
9       0.000000
10      0.000000
11      0.000000
12      0.000000
13      0.000000
14      0.002738
15      0.000000
16      0.000000
17      0.000000
18      0.000000
19      0.008214
20      0.000000
21      0.010951
22      0.000000
23      0.000000
24      0.000000
25      0.000000
26      0.000000
27      0.000000
28      0.000000
29      0.000000
          ...   
3893    1.065024
3894    0.977413
3895    1.867214
3896    0.605065
3897    0.509240
3898    1.037645
3899    0.892539
3900    0.692676
3901    0.624230
3902    1.275838
3903    0.509240
3904    0.509240
3905    0.878850
3906    1.226557
3907    1.264887
3908    0.555784
3909    0.509240
3910    0.958248
3911    1.133470
3912    1.727584
3913    0.859685
3914    2.168378
3915    0.501027
3916    1.262149
3917    0.550308
3918    1.133470
3919    1.149897
3920    0.9418